In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2021
mmdd_str = today.strftime('%m%d')
mmdd_str

'0125'

In [2]:
#today = date(2022, 1, 7)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0125'

In [3]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  2


In [4]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  COTTO
Year:  2021
Quarter:  1
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220
Published:  Tue, 25 Jan 2022 13:26:24 +0700

----------------------------------

F45: 1
Title:  SCGP
Year:  2021
Quarter:  S
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050
Published:  Tue, 25 Jan 2022 12:41:22 +0700


In [5]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']].head()

,name,year,quarter,published
0,COTTO,2021,1,"Tue, 25 Jan 2022 13:26:24 +0700"
1,SCGP,2021,S,"Tue, 25 Jan 2022 12:41:22 +0700"


In [7]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'S','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)

df

,name,year,quarter,link,published
0,COTTO,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,"Tue, 25 Jan 2022 13:26:24 +0700"
1,SCGP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,"Tue, 25 Jan 2022 12:41:22 +0700"


In [8]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df

,name,year,quarter,link,published
0,COTTO,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,"Tue, 25 Jan 2022 13:26:24 +0700"
1,SCGP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,"Tue, 25 Jan 2022 12:41:22 +0700"


In [9]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [10]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [11]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(301, 7)

### Tickers that won't be input

In [12]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link


In [13]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

0

In [14]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,COTTO,2021.0,4,"Tue, 25 Jan 2022 13:26:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220
1,SCGP,2021.0,4,"Tue, 25 Jan 2022 12:41:22 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050


In [15]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

2

In [16]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [17]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(403, 14)

### There are no ticker records

In [18]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market


In [19]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market


### There are ticker records

In [20]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,COTTO,2021.0,4,"Tue, 25 Jan 2022 13:26:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,710,SET
1,SCGP,2021.0,4,"Tue, 25 Jan 2022 12:41:22 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,713,SET50 / SETCLMV / SETTHSI


In [21]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,COTTO,2021.0,4,"Tue, 25 Jan 2022 13:26:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,710,SET
1,SCGP,2021.0,4,"Tue, 25 Jan 2022 12:41:22 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,713,SET50 / SETCLMV / SETTHSI


In [22]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,COTTO,2021,4,"Tue, 25 Jan 2022 13:26:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,710,SET
1,SCGP,2021,4,"Tue, 25 Jan 2022 12:41:22 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,713,SET50 / SETCLMV / SETTHSI


In [23]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')

In [24]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1018, 14)

In [25]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1020, 19)

### Already input, display profit amt & eps to check with new F45

In [26]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge


In [27]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge


In [28]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
0,COTTO,2021,4,"Tue, 25 Jan 2022 13:26:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220,710.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,SCGP,2021,4,"Tue, 25 Jan 2022 12:41:22 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050,713.0,SET50 / SETCLMV / SETTHSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [29]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0    COTTO 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677703220
1     SCGP 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16430677701050
dtype: object